# Async (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#!import ../nbs/Common.fs

In [ ]:
open Common

## choice

In [ ]:
let choice asyncs = async {
    let e = Event<_> ()
    let cts = new System.Threading.CancellationTokenSource ()
    let fn =
        asyncs
        |> Seq.map (fun a -> async {
            let! x = a
            e.Trigger x
        })
        |> Async.Parallel
        |> Async.Ignore
    Async.Start (fn, cts.Token)
    let! result = Async.AwaitEvent e.Publish
    cts.Cancel ()
    return result
}

## map

In [ ]:
let map fn a = async {
    let! x = a
    return fn x
}

## runWithTimeoutAsync

In [ ]:
let runWithTimeoutAsync (timeout : int) fn =
    let getLocals () = $"timeout: {timeout} / {getLocals ()}"

    let timeoutTask = async {
        do! Async.Sleep timeout
        trace Debug (fun () -> "runWithTimeoutAsync") getLocals
        return None
    }

    let task = fn |> map Some

    [ timeoutTask; task ]
    |> choice

In [ ]:
let runWithTimeout timeout fn =
    fn
    |> runWithTimeoutAsync timeout
    |> Async.RunSynchronously

In [ ]:
//// test

Async.Sleep 60
|> runWithTimeout 10
|> _equal None

16:03:02 #1 [Debug] runWithTimeoutAsync / timeout: 10
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeout 60
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>


## runWithTimeoutStrict

In [ ]:
let runWithTimeoutStrict (timeout : int) fn =
    let getLocals () = $"timeout: {timeout} / {getLocals ()}"

    let timeoutTask = async {
        do! Async.Sleep timeout
        return None, getLocals
    }
    
    let task = async {
        try
            return Async.RunSynchronously (fn, timeout) |> Some, getLocals
        with
        | :? System.TimeoutException as ex ->
            let getLocals () = $"ex: {ex |> printException} / {getLocals ()}"
            return None, getLocals
        | e ->
            trace Error (fun () -> "runWithTimeoutStrict") getLocals
            return raise e
    }

    try
        [| timeoutTask; task |]
        |> Array.map Async.StartAsTask
        |> System.Threading.Tasks.Task.WhenAny
        |> fun task ->
            match task.Result.Result with
            | None, getLocals ->
                trace Debug (fun () -> "runWithTimeoutStrict") getLocals
                None
            | result, _ -> result
    with
    | :? System.AggregateException as ex when
        ex.InnerExceptions
        |> Seq.exists (function :? System.Threading.Tasks.TaskCanceledException -> true | _ -> false)
        ->
        let getLocals () = $"ex: {ex |> printException} / {getLocals ()}"
        trace Warn (fun () -> "runWithTimeoutStrict") getLocals
        None
    | ex ->
        trace Error (fun () -> "runWithTimeoutStrict") getLocals
        raise ex

In [ ]:
//// test

Async.Sleep 60
|> runWithTimeoutStrict 10
|> _equal None

16:03:03 #2 [Debug] runWithTimeoutStrict / timeout: 10
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeoutStrict 60
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>


## awaitValueTask

In [ ]:
let awaitValueTaskUnit (task : System.Threading.Tasks.ValueTask) =
    task.AsTask () |> Async.AwaitTask

let awaitValueTask (task : System.Threading.Tasks.ValueTask<_>) =
    task.AsTask () |> Async.AwaitTask

## init

In [ ]:
let init x = async {
    return x
}

In [ ]:
//// test

init 1
|> Async.RunSynchronously
|> _equal 1

1
